# Перед началом работы

1. инструкция по установке

https://www.tensorflow.org/install/

Полезные ссылки:

https://web.stanford.edu/class/cs20si/syllabus.html

https://www.tensorflow.org/programmers_guide/

# Приступим

In [ ]:
import tensorflow as tf

попробуем сложить два числа

In [ ]:
a = tf.add(1,2)

и посмотрим результат

In [ ]:
print (a)

в результате мы получам не число, а ссылку на узел в графе вычеслений

Все переменные вычисляются в сессии - и только в том случае, если вы указали, что хотите посчитать их. До вычисления переменных только граф вычислений. Промежуточные результаты не вычисляются

In [ ]:
sess = tf.Session()
a_res = sess.run(a)
sess.close()

посмотрим вычесленное значение узла графа

In [ ]:
print (a_res)

не пишите так!! вы заменяете ссылку на узел графа вычислений на число.

In [ ]:
with tf.Session() as sess:
    a = sess.run(a)
#     a_res = sess.run(a)
    print (a)

Построим более сложный граф вычислений и посмотрим на него в TensorBoard

# TensorBoard

в TensorFlow по умолчанию используется дефолтный граф вычислений. Добавим в его более сложный подграф и сохраним его содержимое в папку ./graphs.

In [ ]:
x = 2
y = 3
add_op = tf.add(x, y)
mul_op = tf.multiply(x, y)
useless = tf.multiply(x, add_op)
pow_op = tf.pow(add_op, mul_op)

writer = tf.summary.FileWriter('./graphs', tf.get_default_graph())

Посмотрим с помощью следующей команды, запущенной из консоли: 

tensorboard --logdir="./graphs" --port 6006

Далее открываем localhost:6006

И смотрим:)

Можно заметить, что названия вершин генерируются автоматически исходя из типов операций в этой вершине. Для лучшей читаемости графа лучше называть переменные самому. В примеру именеим слоя: input, conv, linear и т.п.

In [ ]:
a = tf.constant(2, name='a')
b = tf.constant(3, name='b')
x = tf.add(a, b, name='add')

x = tf.constant(2, name='input_word')
y = tf.constant(2, name='input_char')
add_op = tf.add(x, y)
mul_op = tf.multiply(x, y)
useless = tf.multiply(x, add_op)
pow_op = tf.pow(add_op, mul_op, name='output')

writer = tf.summary.FileWriter('./graphs', tf.get_default_graph())

# Variables


В TensorFlow вы имеете дело со следующими типами переменных

In [ ]:
a = tf.constant([1, 2], name='a')
b = tf.constant([[0, 1], [2, 3]], name='b')

In [ ]:
s = tf.Variable(2, name="scalar")
m = tf.Variable([[0, 1], [2, 3]], name="matrix") 
W = tf.Variable(tf.zeros([784,10]))
W_constant = tf.Variable(tf.zeros([784,10]), trainable=False)

Для создания изменяемых переменных используйте get_variable. Она имеет доступ в пространству имен и не позволит созвать вам уже существующую переменную с таким именем.

In [ ]:
s = tf.get_variable("scalar", initializer=tf.constant(2))
m = tf.get_variable("matrix", initializer=tf.constant([[0, 1], [2, 3]]))
W = tf.get_variable("big_matrix", shape=(784, 10), initializer=tf.zeros_initializer())

tf.Variable - это класс со множеством доступных операций

x = tf.Variable(...) 

x.initializer # init op

x.value() # read op

x.assign(...) # write op

x.assign_add(...) # and more

Перед вычислением нужно инициализировать переменные заданными значениями

In [ ]:
with tf.Session() as sess:
    print(sess.run(W))

Можно это сделать любым из трех приведённых ниже способов

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(W))
    
# with tf.Session() as sess:
#     sess.run(tf.variables_initializer([a, b]))
    
# W = tf.Variable(tf.zeros([784,10]))
# with tf.Session() as sess:
#     sess.run(W.initializer)

Вычилить значение переменной можно с помощью функции eval

In [ ]:
W = tf.Variable(tf.zeros([784,10]))
with tf.Session() as sess:
    sess.run(W.initializer)
    print(W)
    print (W.eval())


Также следите, какие именно узлы графа вы исполняете

In [ ]:
s = tf.Variable(2, name="scalar")
s.assign(100)
with tf.Session() as sess:
    sess.run(s.initializer)
    print(s.eval())

In [ ]:
s = tf.Variable(2, name="scalar")
s_op = s.assign(100)
with tf.Session() as sess:
    sess.run(s.initializer)
    sess.run(s_op)
    print(s.eval())

# Placeholder

При обучении мы каждый раз подаем новые значения входных данных и меток классов. Существует специальная переменная, в которую можно подавать на вход данные, которую не нужно инициализировать.

In [ ]:
# create a placeholder for a vector of 3 elements, type tf.float32
a = tf.placeholder(tf.float32, shape=[3])

b = tf.constant([5, 5, 5], tf.float32)

# use the placeholder as you would a constant or a variable
c = a + b  # short for tf.add(a, b)

# with tf.Session() as sess:
#     print(sess.run(c))
    
    
with tf.Session() as sess:
    print(sess.run(c, {a: [2, 2, 3]}))


# IF

if в при создании графа вычислений должен зависеть от параметров, не меняющихся в процессе вычисления графа.

# Lazy loading

Выночите все в специальные узлы графа

In [ ]:
x = tf.Variable(10, name='x')
y = tf.Variable(20, name='y')
z = tf.add(x, y)

writer = tf.summary.FileWriter('./graphs/normal_loading', tf.get_default_graph())
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for _ in range(10):
        sess.run(z)

Не делайте следующим образом (пример ниже). Не вычисляйте опериции - всегда вычисляйте узел графа. Или вы будете каджый раз делать новую инициализациию этой операции. Узел графа же инициализируется только один раз.

In [ ]:
x = tf.Variable(10, name='x')
y = tf.Variable(20, name='y')

writer = tf.summary.FileWriter('./graphs/normal_loading', tf.get_default_graph())
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for _ in range(10):
        sess.run(tf.add(x, y))